In [6]:
import pandas as pd
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.decomposition import PCA
import gdown

# Reemplaza 'your_file_id' con el ID de tu archivo en Google Drive
file_id = '1lnKQCUtNXt5cmtWbOBlV1cvVxDvS5-Ed'

# Construye el enlace de descarga directa desde Google Drive
url = f'https://drive.google.com/uc?id={file_id}'

# Descarga el archivo usando gdown
output = 'archivo_csv_modificado.csv'
gdown.download(url, output, quiet=False)

# Guarda el archivo en tu directorio local
output = 'data.csv'
gdown.download(url, output, quiet=False)

# Carga el archivo CSV como un DataFrame de pandas
EmpleadosAttrition = pd.read_csv(output)

# Ahora puedes trabajar con tu DataFrame
print(EmpleadosAttrition.head())

# Convertir variables categóricas en binarias
df = pd.get_dummies(EmpleadosAttrition, columns=['Attrition'], drop_first=True)

#Eliminar columnas que no sirven

# Eliminar varias columnas
columnas_a_eliminar = ['EmployeeCount', 'EmployeeNumber', 'Over18', 'StandardHours']
df.drop(columnas_a_eliminar, axis=1, inplace=True)

# Guardar las columnas binarias en un nuevo DataFrame
EmpleadosAttrition = df.copy()

# Separar las características de la variable de salida
X = EmpleadosAttrition.drop(columns=['Attrition_Yes'])  # Características
y = EmpleadosAttrition['Attrition_Yes']  # Variable de salida (asumiendo 'Attrition_Yes' como la columna binaria de salida)

# Convertir características categóricas en numéricas
X = pd.get_dummies(X)

# Seleccionar las mejores características utilizando SelectKBest
k_best = SelectKBest(score_func=chi2, k=5)  # Utiliza la prueba chi-cuadrado y selecciona las 5 mejores características
X_best = k_best.fit_transform(X, y)

# Obtener las características seleccionadas
caracteristicas_seleccionadas = X.columns[k_best.get_support()]

# Mostrar las características seleccionadas
print("Características seleccionadas:")
print(caracteristicas_seleccionadas)

# Convertir la columna 'HiringDate' al formato de fecha
df['HiringDate'] = pd.to_datetime(df['HiringDate'])

# Obtener la fecha actual
fecha_actual = pd.to_datetime('today')

# Calcular la diferencia en años entre la fecha actual y la fecha de contratación
df['YearsAtCompany'] = (fecha_actual - df['HiringDate']).dt.days // 365

# Mostrar el DataFrame con la nueva columna 'YearsAtCompany'
print(df)

#Eliminar valores NaN
# Eliminar filas con fechas de contratación faltantes o inválidas
df.dropna(subset=['HiringDate'], inplace=True)

# Convertir la columna 'HiringDate' al formato de fecha
df['HiringDate'] = pd.to_datetime(df['HiringDate'])

# Crear la columna 'Year' y obtener el año de contratación
df['Year'] = df['HiringDate'].dt.year.astype(int)

# Mostrar las primeras filas del DataFrame con la nueva columna 'Year'
print(df[['HiringDate', 'Year']].head())

# Paso 5
# Crear la columna 'Year' y obtener el año de contratación
df['Year'] = df['HiringDate'].dt.year.astype(int)

# Mostrar las primeras filas del DataFrame con la nueva columna 'Year'
print(df[['HiringDate', 'Year']].head())

#Paso 6
# Calcular los años que el empleado lleva en la compañía hasta el año 2018
df['YearsAtCompany'] = 2018 - df['Year']

# Mostrar las primeras filas del DataFrame con la nueva columna 'YearsAtCompany'
print(df[['Year', 'YearsAtCompany']].head())

#Paso 8
# Renombrar la variable DistanceFromHome a DistanceFromHome_km
df.rename(columns={'DistanceFromHome': 'DistanceFromHome_km'}, inplace=True)

# Mostrar el DataFrame con el nombre de columna actualizado
print(df.head())

#Paso 9
# Crear la nueva variable DistanceFromHome como entera
df['DistanceFromHome'] = df['DistanceFromHome_km'].str.extract('(\d+)').astype(int)

# Mostrar el DataFrame con la nueva variable DistanceFromHome
print(df.head())

#Paso 10

# Eliminar las columnas Year, HiringDate y DistanceFromHome_km
df.drop(['Year', 'HiringDate', 'DistanceFromHome_km'], axis=1, inplace=True)

# Mostrar el DataFrame después de eliminar las columnas
print(df.head())

#paso 11
# Crear el DataFrame SueldoPromedioDepto con el salario mensual promedio por departamento
SueldoPromedioDepto = df.groupby('Department')['MonthlyIncome'].mean().reset_index()

# Renombrar la columna MonthlyIncome a SueldoPromedio
SueldoPromedioDepto.rename(columns={'MonthlyIncome': 'SueldoPromedio'}, inplace=True)

# Mostrar el nuevo DataFrame
print(SueldoPromedioDepto)
#PASO 12
from sklearn.preprocessing import MinMaxScaler

# Inicializar el escalador
scaler = MinMaxScaler()

# Escalar la variable MonthlyIncome
df['MonthlyIncome_scaled'] = scaler.fit_transform(df[['MonthlyIncome']])

# Mostrar las primeras filas del DataFrame con la variable escalada
print(df[['MonthlyIncome', 'MonthlyIncome_scaled']].head())
#PASO 13
# Convertir variables categóricas a numéricas utilizando one-hot encoding
df = pd.get_dummies(df, columns=['BusinessTravel', 'Department', 'EducationField', 'Gender', 'JobRole', 'MaritalStatus', 'Attrition_Yes'], drop_first=True)

# Mostrar el DataFrame con las variables convertidas
print(df.head())

#PASO 14 CORRELACIÓN
# Convertir variables categóricas en numéricas
df = pd.get_dummies(df)

correlation = df.corr()['Attrition_Yes_1'].sort_values(ascending=False)
print(correlation)

#PASO #15
# Seleccionar variables con correlación mayor o igual a 0.1
correlated_features = correlation[correlation >= 0.1].index.tolist()

# Incluir la variable de salida 'Attrition_Yes'
correlated_features.append('Attrition_Yes_1')

# Crear un nuevo DataFrame con las variables seleccionadas
EmpleadosAttritionFinal = df[correlated_features]

# nuevo DataFrame
print(EmpleadosAttritionFinal.head())

#PASO 16
# PCA COMPONENTES
pca = PCA(n_components=2)

# Ajustar y transformar los datos de EmpleadosAttritionFinal utilizando PCA
EmpleadosAttritionPCA = pca.fit_transform(EmpleadosAttritionFinal)

# Convertir el resultado de PCA a un DataFrame
EmpleadosAttritionPCA = pd.DataFrame(EmpleadosAttritionPCA, columns=['Componente1', 'Componente2'])

# Mostrar las primeras filas del nuevo DataFrame EmpleadosAttritionPCA
print(EmpleadosAttritionPCA.head())

#PASO 17
# Inicializar PCA con el número de componentes suficientes para explicar el 80% de la varianza
pca = PCA(n_components=0.8)

# Aplicar PCA al dataframe EmpleadosAttritionFinal
EmpleadosAttritionPCA = pca.fit_transform(EmpleadosAttritionFinal)

# Convertir el array resultante de PCA a un DataFrame
EmpleadosAttritionPCA = pd.DataFrame(data=EmpleadosAttritionPCA, columns=[f'Componente_{i}' for i in range(1, pca.n_components_ + 1)])

# Concatenar EmpleadosAttritionPCA con la columna Attrition del dataframe original
EmpleadosAttritionFinal = pd.concat([EmpleadosAttritionPCA, EmpleadosAttritionFinal['Attrition_Yes_1']], axis=1)
print(EmpleadosAttritionFinal.head())

# Guardar el DataFrame EmpleadosAttritionFinal en un archivo CSV
EmpleadosAttritionFinal.to_csv('EmpleadosAttritionFinal.csv', index=False)




Downloading...
From: https://drive.google.com/uc?id=1lnKQCUtNXt5cmtWbOBlV1cvVxDvS5-Ed
To: /content/archivo_csv_modificado.csv
100%|██████████| 59.8k/59.8k [00:00<00:00, 58.3MB/s]
Downloading...
From: https://drive.google.com/uc?id=1lnKQCUtNXt5cmtWbOBlV1cvVxDvS5-Ed
To: /content/data.csv
100%|██████████| 59.8k/59.8k [00:00<00:00, 18.8MB/s]


   Age BusinessTravel              Department DistanceFromHome  Education  \
0   50  Travel_Rarely  Research & Development             1 km          2   
1   36  Travel_Rarely  Research & Development             6 km          2   
2   21  Travel_Rarely                   Sales             7 km          1   
3   52  Travel_Rarely  Research & Development             7 km          4   
4   33  Travel_Rarely  Research & Development            15 km          1   

  EducationField  EmployeeCount  EmployeeNumber  EnvironmentSatisfaction  \
0        Medical              1             997                        4   
1        Medical              1             178                        2   
2      Marketing              1            1780                        2   
3  Life Sciences              1            1118                        2   
4        Medical              1             582                        2   

  Gender  ...  PercentSalaryHike  PerformanceRating RelationshipSatisfaction  \
